# Scraping des publications ArXiv liées à Quandela

Ce notebook interroge l'API ArXiv pour récupérer les publications contenant le mot-clé **"Quandela"** (entreprise de calcul quantique) et en extraire un maximum d'informations (titre, auteurs, résumé, DOI, etc.).

Objectifs :
- **Récupérer toutes les publications ArXiv récentes contenant le mot-clé `Quandela`**.
- **Extraire les métadonnées utiles** (titre, auteurs, dates, catégories, DOI, journal, résumé…).
- **Filtrer par années récentes** (par exemple à partir de 2020).
- **Exporter les résultats** dans un format exploitable (CSV / DataFrame Pandas).


In [1]:
import requests
import xml.etree.ElementTree as ET
from collections import Counter
import re
import pandas as pd
from typing import List, Dict

# -----------------------------
# Paramètres généraux
# -----------------------------
AFFILIATION_KEYWORD = "quandela"  # mot-clé lié à l'entreprise
MAX_RESULTS = 2000                 # nombre total max de résultats voulus
MAX_RESULTS_PER_PAGE = 100         # limite ArXiv par requête (1000 max, on reste plus bas par prudence)
BASE_URL = "http://export.arxiv.org/api/query"

# Filtre temporel (par exemple : ne garder que les papiers publiés à partir de 2020)
MIN_YEAR = 2020

print(f"Mot-clé utilisé : {AFFILIATION_KEYWORD}")
print(f"Nombre maximum de résultats attendus : {MAX_RESULTS}")
print(f"Année minimale : {MIN_YEAR}")


Mot-clé utilisé : quandela
Nombre maximum de résultats attendus : 2000
Année minimale : 2020


In [2]:
# -----------------------------
# Fonctions utilitaires pour ArXiv
# -----------------------------

NAMESPACE = {
    "atom": "http://www.w3.org/2005/Atom",
    "opensearch": "http://a9.com/-/spec/opensearch/1.1/"
}


def build_query_url(keyword: str, start: int, max_results: int) -> str:
    """Construit l'URL de requête pour l'API ArXiv avec pagination."""
    query = f"all:{keyword}"
    return (
        f"{BASE_URL}?search_query={query}"
        f"&start={start}&max_results={max_results}"
    )


def fetch_arxiv_batch(keyword: str, start: int, max_results: int = MAX_RESULTS_PER_PAGE) -> ET.Element:
    """Récupère un batch de résultats ArXiv et retourne la racine XML."""
    url = build_query_url(keyword, start, max_results)
    print(f"Requête ArXiv: start={start}, max_results={max_results}")
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    root = ET.fromstring(response.text)
    return root


def parse_entry(entry: ET.Element) -> Dict:
    """Extrait les champs intéressants d'une entrée Atom ArXiv."""
    # Dates
    published_text = entry.find("atom:published", NAMESPACE)
    updated_text = entry.find("atom:updated", NAMESPACE)
    published = published_text.text if published_text is not None else None
    updated = updated_text.text if updated_text is not None else None

    # Année
    year = None
    if published:
        m = re.match(r"(\d{4})-", published)
        if m:
            year = int(m.group(1))

    # Titre
    title_el = entry.find("atom:title", NAMESPACE)
    title = title_el.text.strip().replace("\n", " ") if title_el is not None else None

    # Résumé
    summary_el = entry.find("atom:summary", NAMESPACE)
    summary = summary_el.text.strip().replace("\n", " ") if summary_el is not None else None

    # Auteurs (liste de noms)
    authors = [
        a.find("atom:name", NAMESPACE).text.strip()
        for a in entry.findall("atom:author", NAMESPACE)
        if a.find("atom:name", NAMESPACE) is not None
    ]

    # ID ArXiv (URL) et identifiant court
    id_el = entry.find("atom:id", NAMESPACE)
    id_url = id_el.text if id_el is not None else None
    arxiv_id = None
    if id_url and "arxiv.org/abs/" in id_url:
        arxiv_id = id_url.split("arxiv.org/abs/")[-1]

    # DOI
    doi = None
    for link in entry.findall("atom:link", NAMESPACE):
        if link.get("title") == "doi":
            doi = link.get("href")

    # Journal ref & catégories
    journal_ref_el = entry.find("atom:arxiv:journal_ref", {**NAMESPACE, "arxiv": "http://arxiv.org/schemas/atom"})
    journal_ref = journal_ref_el.text if journal_ref_el is not None else None

    categories = [c.get("term") for c in entry.findall("atom:category", NAMESPACE) if c.get("term")]

    return {
        "arxiv_id": arxiv_id,
        "id_url": id_url,
        "title": title,
        "summary": summary,
        "authors": ", ".join(authors),
        "published": published,
        "updated": updated,
        "year": year,
        "doi": doi,
        "journal_ref": journal_ref,
        "categories": ", ".join(categories),
    }


def fetch_all_results(keyword: str, max_results: int = MAX_RESULTS) -> List[Dict]:
    """Itère sur l'API ArXiv pour récupérer jusqu'à max_results entrées."""
    entries_data: List[Dict] = []
    start = 0

    while start < max_results:
        root = fetch_arxiv_batch(keyword, start, min(MAX_RESULTS_PER_PAGE, max_results - start))
        batch_entries = root.findall("atom:entry", NAMESPACE)

        if not batch_entries:
            print("Plus de résultats retournés par l'API.")
            break

        for entry in batch_entries:
            data = parse_entry(entry)
            entries_data.append(data)

        print(f"Total cumulé d'entrées : {len(entries_data)}")

        # Arrêt si on a dépassé max_results ou si le batch est plus petit que la taille demandée
        if len(batch_entries) < min(MAX_RESULTS_PER_PAGE, max_results - start):
            break

        start += MAX_RESULTS_PER_PAGE

    return entries_data


In [3]:
# -----------------------------
# Récupération brute des données ArXiv
# -----------------------------

raw_entries = fetch_all_results(AFFILIATION_KEYWORD, max_results=MAX_RESULTS)

print(f"\nNombre total de résultats récupérés : {len(raw_entries)}")

# Conversion en DataFrame
arxiv_df = pd.DataFrame(raw_entries)

print("\nAperçu des colonnes :")
print(arxiv_df.columns.tolist())

arxiv_df.head()


Requête ArXiv: start=0, max_results=100
Total cumulé d'entrées : 8

Nombre total de résultats récupérés : 8

Aperçu des colonnes :
['arxiv_id', 'id_url', 'title', 'summary', 'authors', 'published', 'updated', 'year', 'doi', 'journal_ref', 'categories']


,arxiv_id,id_url,title,summary,authors,published,updated,year,doi,journal_ref,categories
0,2404.12729v2,http://arxiv.org/abs/2404.12729v2,Demonstration of quantum projective simulation...,Variational quantum algorithms show potential ...,"Giacomo Franceschetto, Arno Ricou",2024-04-19T09:17:15Z,2024-11-06T16:47:22Z,2024,https://doi.org/10.1103/PhysRevA.110.062613,None,quant-ph
1,2509.04266v2,http://arxiv.org/abs/2509.04266v2,Foundations of photonic quantum computation,This work aims to introduce the fundamental co...,"Martin Bombardelli, Gerard Fleury, Philippe La...",2025-09-04T14:43:10Z,2025-10-31T13:27:40Z,2025,None,None,quant-ph
2,2301.09594v2,http://arxiv.org/abs/2301.09594v2,Solving graph problems with single-photons and...,An important challenge for current and near-te...,"Rawad Mezher, Ana Filipa Carvalho, Shane Mansf...",2023-01-23T17:48:33Z,2023-08-14T17:15:13Z,2023,None,None,quant-ph
3,2411.03878v1,http://arxiv.org/abs/2411.03878v1,Quantum circuit compression using qubit logic ...,"We present qubit logic on qudits (QLOQ), a com...","Liam Lysaght, Timothée Goubault, Patrick Sinno...",2024-11-06T12:49:32Z,2024-11-06T12:49:32Z,2024,None,None,quant-ph
4,2510.14746v1,http://arxiv.org/abs/2510.14746v1,Quantum remeshing and efficient encoding for f...,We present a variational quantum algorithm for...,"Ulysse Remond, Pierre-Emmanuel Emeriau, Liam L...",2025-10-16T14:50:59Z,2025-10-16T14:50:59Z,2025,None,None,quant-ph


In [4]:
# -----------------------------
# Filtrage temporel + statistiques par année
# -----------------------------

# On enlève les lignes sans année
arxiv_df_clean = arxiv_df.dropna(subset=["year"]).copy()

# Filtre à partir de MIN_YEAR
recent_df = arxiv_df_clean[arxiv_df_clean["year"] >= MIN_YEAR].copy()

print(f"Nombre de publications avec année connue : {len(arxiv_df_clean)}")
print(f"Nombre de publications à partir de {MIN_YEAR} : {len(recent_df)}")

# Comptage par année
counts = recent_df["year"].value_counts().sort_index()

print("\n--------------------------------------------")
print(" Publications ArXiv contenant le mot-clé 'Quandela' (filtrées)")
print("--------------------------------------------")

total = int(counts.sum())
for year, count in counts.items():
    print(f" {year} : {count} publications")

print("--------------------------------------------")
print(f"TOTAL : {total} publications")
print("--------------------------------------------")

recent_df.head()


Nombre de publications avec année connue : 8
Nombre de publications à partir de 2020 : 8

--------------------------------------------
 Publications ArXiv contenant le mot-clé 'Quandela' (filtrées)
--------------------------------------------
 2023 : 2 publications
 2024 : 2 publications
 2025 : 4 publications
--------------------------------------------
TOTAL : 8 publications
--------------------------------------------


,arxiv_id,id_url,title,summary,authors,published,updated,year,doi,journal_ref,categories
0,2404.12729v2,http://arxiv.org/abs/2404.12729v2,Demonstration of quantum projective simulation...,Variational quantum algorithms show potential ...,"Giacomo Franceschetto, Arno Ricou",2024-04-19T09:17:15Z,2024-11-06T16:47:22Z,2024,https://doi.org/10.1103/PhysRevA.110.062613,None,quant-ph
1,2509.04266v2,http://arxiv.org/abs/2509.04266v2,Foundations of photonic quantum computation,This work aims to introduce the fundamental co...,"Martin Bombardelli, Gerard Fleury, Philippe La...",2025-09-04T14:43:10Z,2025-10-31T13:27:40Z,2025,None,None,quant-ph
2,2301.09594v2,http://arxiv.org/abs/2301.09594v2,Solving graph problems with single-photons and...,An important challenge for current and near-te...,"Rawad Mezher, Ana Filipa Carvalho, Shane Mansf...",2023-01-23T17:48:33Z,2023-08-14T17:15:13Z,2023,None,None,quant-ph
3,2411.03878v1,http://arxiv.org/abs/2411.03878v1,Quantum circuit compression using qubit logic ...,"We present qubit logic on qudits (QLOQ), a com...","Liam Lysaght, Timothée Goubault, Patrick Sinno...",2024-11-06T12:49:32Z,2024-11-06T12:49:32Z,2024,None,None,quant-ph
4,2510.14746v1,http://arxiv.org/abs/2510.14746v1,Quantum remeshing and efficient encoding for f...,We present a variational quantum algorithm for...,"Ulysse Remond, Pierre-Emmanuel Emeriau, Liam L...",2025-10-16T14:50:59Z,2025-10-16T14:50:59Z,2025,None,None,quant-ph


In [5]:
# -----------------------------
# Export des résultats
# -----------------------------

output_csv = "arxiv_quandela_publications.csv"
recent_df.to_csv(output_csv, index=False)

print(f"Fichier CSV exporté : {output_csv}")
print(f"Nombre de lignes exportées : {len(recent_df)}")


Fichier CSV exporté : arxiv_quandela_publications.csv
Nombre de lignes exportées : 8
